## Facial keypoints detection

In this task you will create facial keypoint detector based on CNN regressor.


![title](example.png)

### Load and preprocess data

Script `get_data.py` unpacks data — images and labelled points. 6000 images are located in `images` folder and keypoint coordinates are in `gt.csv` file. Run the cell below to unpack data.

In [ ]:
from get_data import unpack
from pathlib import Path
cur_dir = Path('.').absolute()
data_dir = cur_dir.joinpath('data')
if not data_dir.is_dir():
    unpack('data.tar.xz', cur_dir)
else:
    print('"data" is already unpacked')

Now you have to read `gt.csv` file and images from `images` dir. File `gt.csv` contains header and ground truth points for every image in `images` folder. It has 29 columns. First column is a filename and next 28 columns are `x` and `y` coordinates for 14 facepoints. We will make following preprocessing:
1. Scale all images to resolution $100 \times 100$ pixels.
2. Scale all coordinates to range $[-0.5; 0.5]$. To obtain that, divide all x's by width (or number of columns) of image, and divide all y's by height (or number of rows) of image and subtract 0.5 from all values.

Function `load_imgs_and_keypoint` should return a tuple of two numpy arrays: `imgs` of shape `(N, 100, 100, 3)`, where `N` is the number of images and `points` of shape `(N, 28)`.

In [ ]:
import random
import numpy as np
import pandas as pd
from skimage.color import gray2rgb
from skimage.color import rgb2gray
from skimage.io import imread
from skimage.transform import resize
from tqdm import tqdm_notebook
import gc

In [ ]:
random.seed(123)
np.random.seed(123)

In [ ]:
def get_imgs(dirname):
    """
    Obtain images from dirname
    
    Parameters
    ----------
    dirname : Path or str
        The directory to obtain the images from
    
    Returns
    -------
    imgs : np.array, shape (N, 100, 100, 3)
        The N images in the dirname directory
    orig_width : np.array, shape (N,)
        The original widths of the images
    orig_height : np.array, shape (N,)
        The original heights of the images
    """
    
    # Sorting to let the filename correspond with points
    img_paths = sorted(dirname.glob('**/*.jpg'))
    n_imgs = len(img_paths)
    size = 100
    
    # Define the shape of imgs
    imgs = np.zeros((n_imgs, size, size, 3))
    orig_width = np.zeros(n_imgs)
    orig_height = np.zeros(n_imgs)
    
    for nr, img_path in enumerate(tqdm_notebook(img_paths)):
        img = imread(img_path)
        # Ensure 3 channels
        if len(img.shape) == 2:
            img = gray2rgb(img)
            
        # Store width and height
        orig_width[nr] = img.shape[0]
        orig_height[nr] = img.shape[1]
        
        # Resize and store
        img = resize(img, (size, size), mode='reflect', anti_aliasing=True)
        imgs[nr, :, :, :] = img
        
    return imgs, orig_width, orig_height

In [ ]:
def get_points(dirname, orig_width, orig_height):
    """
    Obtain points from dirname
    
    Parameters
    ----------
    dirname : Path or str
        The directory to obtain the points from
    
    Returns
    -------
    points : np.array, shape (N, 28)
        Keypoints belonging to corresponding images
    orig_width : np.array, shape (N,)
        The original widths of the images
    orig_height : np.array, shape (N,)
        The original heights of the images
    """
    
    csv_path = data_dir.joinpath('gt.csv')
    points_df = pd.read_csv(csv_path)
    # NOTE: The filename is sorted in the same manner as imgs are
    # NOTE: Casting to float to use the /= operator
    points = points_df.loc[:, [col for col in points_df.columns if col != 'filename']].values.astype(float)

    # Normalize
    for i in tqdm_notebook(range(points.shape[0])):
        # NOTE: The columns are arranged like the following
        #       x1 y1 x2 x2 ... x14 y14
        # Normalize width (the xs) by taking every second column
        points[i, ::2] /= orig_width[i]
        # Normalize height (the ys) by taking every second column starting from 1
        points[i, 1::2] /= orig_height[i]
    
    # Scale to [-0.5, 0.5]
    points -= 0.5
    
    return points

In [ ]:
def load_imgs_and_keypoints(dirname=cur_dir.joinpath('data')):
    """
    Loads the images and keypoints
    
    Parameters
    ----------
    dirname : Path or str
        The directory to obtain the data from
    
    Returns
    -------
    imgs : np.array, shape (N, 100, 100, 3)
        The N images in the dirname directory
    points : np.array, shape (N, 28)
        Keypoints belonging to imgs
    """
    
    imgs, orig_width, orig_height = get_imgs(dirname)
    points = get_points(dirname, orig_width, orig_height)

    return imgs, points

imgs, points = load_imgs_and_keypoints()

In [ ]:
print(f'Max {points.max():.2f} found at image {np.where(np.isclose(points, points.max()))[0][0]}')
print(f'Min {points.min():.2f} found at image {np.where(np.isclose(points, points.min()))[0][0]}')

> **NOTE**: By inspecting the images with the max and the min, we indeed find that these points are outside the frame.

In [ ]:
# Example of output
%matplotlib inline
from skimage.io import imshow
imshow(imgs[0])
points[0]

### Visualize data

Let's prepare a function to visualize points on image. Such function obtains two arguments: an image and a vector of points' coordinates and draws points on image (just like first image in this notebook).

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import Circle

def visualize_points(img, points):
    """
    Visualizes the points on the image
    
    Parameters
    ----------
    img : np.array, shape (cols, rows)
        The image
    points : np.array, shape (28)
        The points given like x1 y1 x2 y2 ... x14 y14
    """
    # Make point pairs
    point_pairs = list(zip(points[::2], points[1::2]))
    
    fig, ax = plt.subplots()
    # Plot the image
    ax.imshow(img)
    # Plot the points
    for x, y in point_pairs:
        # Backtransform
        x = (x+0.5)*100
        y = (y+0.5)*100
        # Plot
        circle = plt.Circle((x, y), radius=1, color='r')
        ax.add_artist(circle)
    
visualize_points(imgs[1], points[1])

### Train/val split

Run the following code to obtain train/validation split for training neural network.

In [ ]:
from sklearn.model_selection import train_test_split
imgs_train, imgs_val, points_train, points_val = train_test_split(imgs, points, test_size=0.1)

### Simple data augmentation

For better training we will use simple data augmentation — flipping an image and points. Implement function flip_img which flips an image and its' points. Make sure that points are flipped correctly! For instance, points on right eye now should be points on left eye (i.e. you have to mirror coordinates and swap corresponding points on the left and right sides of the face). Visualize an example of original and flipped image.

In [ ]:
def flip_img(img, points):
    """
    Flips and image and its points
    
    Parameters
    ----------
    img : np.array, shape (100, 100)
        The image to flip
    points : np.array, shape (28)
        The points given like x1 y1 x2 y2 ... x14 y14
        
    Returns
    -------
    f_img : np.array, shape (100, 100)
        The flipped image
    f_points : np.array, shape (100, 100)
        The flipped points given like x1 y1 x2 y2 ... x14 y14
    """
    
    # Flip the image
    f_img = img.copy()
    # Flipping the x-axis will simply be to reverse the column order
    # NOTE: We use ellipsis to have arbitrary numbers of first dimensions
    # https://stackoverflow.com/questions/772124/what-does-the-python-ellipsis-object-do
    f_img = f_img[..., ::-1, :]
    
    # Flip the points along the x-axis by negating the x coordinate
    f_points = points.copy()
    # NOTE: We use ellipsis to have arbitrary numbers of first dimensions
    f_points[..., ::2] = -f_points[..., ::2] 
    
    return f_img, f_points

f_img, f_points = flip_img(imgs[1], points[1])
visualize_points(f_img, f_points)

Time to augment our training sample. Apply flip to every image in training sample. As a result you should obtain two arrays: `aug_imgs_train` and `aug_points_train` which contain original images and points along with flipped ones.

In [ ]:
aug_imgs_train, aug_points_train = flip_img(imgs_train, points_train)

In [ ]:
visualize_points(aug_imgs_train[2], aug_points_train[2])

In [ ]:
visualize_points(aug_imgs_train[3], aug_points_train[3])

In [ ]:
# Make training data and validation data
x_train = np.concatenate((imgs_train, aug_imgs_train), axis=0)/255
x_val = imgs_val/255

In [ ]:
y_train = np.concatenate((points_train, aug_points_train), axis=0)
y_val = points_val

### Network architecture and training

Now let's define neural network regressor. It will have 28 outputs, 2 numbers per point. The precise architecture is up to you. We recommend to add 2-3 (`Conv2D` + `MaxPooling2D`) pairs, then `Flatten` and 2-3 `Dense` layers. Don't forget about ReLU activations. We also recommend to add `Dropout` to every `Dense` layer (with p from 0.2 to 0.5) to prevent overfitting.


In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU
from keras import regularizers

In [ ]:
def compile_model(model, loss='mse', optimizer='adadelta'):
    """
    Compiles the model
    
    Parameters
    ----------
    model : Sequential
        The uncompiled model
    loss : str or function
        The loss function
    optimizer : str or keras.optimizer
        The optimizer
    
    Returns
    -------
    model : Sequential
        The compiled model
    """
    
    model.compile(loss=loss,
                  optimizer=optimizer,
                  metrics=['mse'])

    return model

In [ ]:
from keras.models import load_model
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
import pickle

def fit_model(model,
              name,
              x_train_=x_train,
              y_train_=y_train,
              x_val_=x_val,
              y_val_=y_val,
              patience=7,
              batch_size=64,
              epochs=100):
    """
    Fits the model
    
    Notes
    -----
    x_train, y_train, x_val and y_val must be global variables
    
    Parameters
    ----------
    model : Sequential
        The compiled model
    name : str
        Name of the model
    x_train_ : array-like
        The training data
    y_train_ :array-like
        The target data for training
    x_val_ : array-like
        The validation data
    y_val_ :array-like
        The target data for validation
    patience : int
        How long to wait for non-imporving validation error
    batch_size : int
        The batch size
    epochs : int
        Number of epochs
    
    Returns
    -------
    model : Sequential
        The fitted model
    history : dict
        The history of the fit
    """
    
    model_dir = Path('.').absolute().joinpath('model')
    model_dir.mkdir(exist_ok=True)
    model_path = model_dir.joinpath(f'{name}.h5')
    history_path = model_dir.joinpath(f'{name}.pickle')
    
    if not model_path.is_file():
        stopper = EarlyStopping(monitor='val_loss', patience=patience, verbose=1, mode='auto', baseline=None)
        checkpointer = ModelCheckpoint(filepath=str(model_path),
                                       verbose=1, 
                                       save_best_only=True)
        history = model.fit(x_train_,
                            y_train_,
                            validation_data=(x_val_, y_val_),
                            batch_size=64,
                            epochs=100,
                            shuffle=True,
                            callbacks=[checkpointer, stopper])
        
        history = history.history
        with history_path.open('wb') as f:
            pickle.dump(history, f, pickle.HIGHEST_PROTOCOL)  
    else:
        model = load_model(str(model_path))
        with history_path.open('rb') as f:
            history = pickle.load(f)
    return model, history

In [ ]:
def plot_history(history):
    """
    Plots the history
    
    Parameters
    ----------
    history : history
        The history of the fit
        
    Returns
    -------
    best_val_mse : float
        The best validation mse
    """
    
    fig, ax = plt.subplots()
    
    # NOTE: When regularizers are used, the loss is no longer the same as mse
    ax.plot(history['mean_squared_error'], label='train_mse')
    ax.plot(history['val_mean_squared_error'], label='val_mse')
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Mean squared error')
    ax.set_yscale('log')
    ax.grid(True)
    _ = ax.legend(loc='lower left')
    
    best_val_mse = np.array(history['val_mean_squared_error']).min()
    
    return best_val_mse

Time to train! Since we are training a regressor, make sure that you use mean squared error (mse) as loss. Feel free to experiment with optimization method (SGD, Adam, etc.) and its' parameters.

In [ ]:
def lenet(input_shape, dropout=0.3, outputs=28):
    """
    Returns an uncompiled lenet-like model
    
    Parameters
    ----------
    input_shape : array-like, shape (3,)
        Tuple containing height, width and depth
    outputs : int
        Number of outputs
    
    Returns
    -------
    model : Sequential
        The uncompiled model
    """
    
    model = Sequential()

    model.add(Conv2D(filters=20,
                     kernel_size=5,
                     strides=1,
                     padding='same',
                     activation='relu',
                     input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=2,
                           strides=2))
    
    model.add(Conv2D(filters=50,
                     kernel_size=5,
                     strides=1,
                     padding='same',
                     activation='relu',
                     input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=2,
                           strides=2))

    model.add(Flatten())
    model.add(Dense(500, activation='relu'))
    model.add(Dropout(dropout))
    
    # NOTE: We use identity on the last layer as we are dealing with a regression problem
    model.add(Dense(outputs, activation='linear'))
    
    model.summary()
    
    return model

In [ ]:
def alexnet(input_shape, dropout=0.3, outputs=28):
    """
    Returns an uncompiled alexnet-like model
    
    Parameters
    ----------
    input_shape : array-like, shape (3,)
        Tuple containing height, width and depth
    outputs : int
        Number of outputs
    
    Returns
    -------
    model : Sequential
        The uncompiled model
    """
    
    model = Sequential()

    model.add(Conv2D(filters=96,
                     kernel_size=11,
                     strides=4,
                     padding='same',
                     activation='relu',
                     input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=3, 
                           strides=2))
    
    model.add(Conv2D(filters=256,
                     kernel_size=5,
                     strides=1,
                     padding='same',
                     activation='relu'))
    model.add(MaxPooling2D(pool_size=3,
                           strides=2))
    
    model.add(Conv2D(filters=384,
                     kernel_size=5,
                     strides=1,
                     padding='same',
                     activation='relu'))
    model.add(Conv2D(filters=384,
                     kernel_size=5,
                     strides=1,
                     padding='same',
                     activation='relu'))
    model.add(Conv2D(filters=256,
                     kernel_size=5,
                     strides=1,
                     padding='same',
                     activation='relu'))
    model.add(MaxPooling2D(pool_size=3,
                           strides=2))
    
    model.add(Flatten())
    
    model.add(Dense(9216, activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(dropout))

    # NOTE: We use identity on the last layer as we are dealing with a regression problem
    model.add(Dense(outputs, activation='linear'))
    
    model.summary()
    
    return model

In [ ]:
def lenet_double(input_shape, dropout=0.3, outputs=28):
    """
    Returns an uncompiled lenet-like with more pooling and more fully connected layers
    
    Parameters
    ----------
    input_shape : array-like, shape (3,)
        Tuple containing height, width and depth
    outputs : int
        Number of outputs
    
    Returns
    -------
    model : Sequential
        The uncompiled model
    """
    
    model = Sequential()

    model.add(Conv2D(filters=20,
                     kernel_size=5,
                     strides=1,
                     padding='same',
                     activation='relu',
                     input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=2,
                           strides=2))
    
    model.add(Conv2D(filters=50,
                     kernel_size=5,
                     strides=1,
                     padding='same',
                     activation='relu',
                     input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=2,
                           strides=2))
    
    model.add(Conv2D(filters=120,
                     kernel_size=5,
                     strides=1,
                     padding='same',
                     activation='relu',
                     input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=2,
                           strides=2))

    model.add(Flatten())
    model.add(Dense(800, activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(500, activation='relu'))
    model.add(Dropout(dropout))
    
    # NOTE: We use identity on the last layer as we are dealing with a regression problem
    model.add(Dense(outputs, activation='linear'))
    
    model.summary()
    
    return model

In [ ]:
def lenet_double_batch_normalization(input_shape, dropout=0.3, outputs=28):
    """
    Returns an uncompiled lenet_double, but with batch normalization
    
    Parameters
    ----------
    input_shape : array-like, shape (3,)
        Tuple containing height, width and depth
    outputs : int
        Number of outputs
    
    Returns
    -------
    model : Sequential
        The uncompiled model
    """
    
    model = Sequential()

    model.add(Conv2D(filters=20,
                     kernel_size=5,
                     strides=1,
                     padding='same',
                     activation='relu',
                     input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=2,
                           strides=2))
    
    model.add(Conv2D(filters=50,
                     kernel_size=5,
                     strides=1,
                     padding='same',
                     activation='relu',
                     input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=2,
                           strides=2))
    
    model.add(Conv2D(filters=120,
                     kernel_size=5,
                     strides=1,
                     padding='same',
                     activation='relu',
                     input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=2,
                           strides=2))

    model.add(Flatten())
    model.add(Dense(800, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(dropout))
    model.add(BatchNormalization())
    model.add(Dense(500, activation='relu'))
    model.add(Dropout(dropout))
    model.add(BatchNormalization())
    
    # NOTE: We use identity on the last layer as we are dealing with a regression problem
    model.add(Dense(outputs, activation='linear'))
    
    model.summary()
    
    return model

In [ ]:
def lenet_double_bn_leaky(input_shape, dropout=0.3, outputs=28):
    """
    Returns an uncompiled lenet_double_batch_normalization, but with leaky ReLU activation
    
    Parameters
    ----------
    input_shape : array-like, shape (3,)
        Tuple containing height, width and depth
    outputs : int
        Number of outputs
    
    Returns
    -------
    model : Sequential
        The uncompiled model
    """
    
    model = Sequential()

    model.add(Conv2D(filters=20,
                     kernel_size=5,
                     strides=1,
                     padding='same',
                     input_shape=input_shape))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=2,
                           strides=2))
    
    model.add(Conv2D(filters=50,
                     kernel_size=5,
                     strides=1,
                     padding='same',
                     input_shape=input_shape))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=2,
                           strides=2))
    
    model.add(Conv2D(filters=120,
                     kernel_size=5,
                     strides=1,
                     padding='same',
                     input_shape=input_shape))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=2,
                           strides=2))

    model.add(Flatten())
    model.add(Dense(800))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(Dropout(dropout))
    model.add(BatchNormalization())
    model.add(Dense(500))
    model.add(LeakyReLU())
    model.add(Dropout(dropout))
    model.add(BatchNormalization())
    
    # NOTE: We use identity on the last layer as we are dealing with a regression problem
    model.add(Dense(outputs, activation='linear'))
    
    model.summary()
    
    return model

In [ ]:
def lenet_d_bn_l_more_filters(input_shape, dropout=0.3, outputs=28):
    """
    Returns an uncompiled lenet_double_bn_leaky, but with more filters
    
    Parameters
    ----------
    input_shape : array-like, shape (3,)
        Tuple containing height, width and depth
    outputs : int
        Number of outputs
    
    Returns
    -------
    model : Sequential
        The uncompiled model
    """
    
    model = Sequential()

    model.add(Conv2D(filters=32,
                     kernel_size=5,
                     strides=1,
                     padding='same',
                     input_shape=input_shape))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=2,
                           strides=2))
    
    model.add(Conv2D(filters=64,
                     kernel_size=5,
                     strides=1,
                     padding='same',
                     input_shape=input_shape))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=2,
                           strides=2))
    
    model.add(Conv2D(filters=256,
                     kernel_size=5,
                     strides=1,
                     padding='same',
                     input_shape=input_shape))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=2,
                           strides=2))

    model.add(Flatten())
    model.add(Dense(800))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(Dropout(dropout))
    model.add(BatchNormalization())
    model.add(Dense(500))
    model.add(LeakyReLU())
    model.add(Dropout(dropout))
    model.add(BatchNormalization())
    
    # NOTE: We use identity on the last layer as we are dealing with a regression problem
    model.add(Dense(outputs, activation='linear'))
    
    model.summary()
    
    return model

In [ ]:
def lenet_d_bn_l_more_filters_reg(input_shape, dropout=0.3, outputs=28):
    """
    Returns an uncompiled lenet_d_bn_l_more_filters, but with regularization in the last layer
    
    Parameters
    ----------
    input_shape : array-like, shape (3,)
        Tuple containing height, width and depth
    outputs : int
        Number of outputs
    
    Returns
    -------
    model : Sequential
        The uncompiled model
    """
    
    model = Sequential()

    model.add(Conv2D(filters=32,
                     kernel_size=5,
                     strides=1,
                     padding='same',
                     input_shape=input_shape))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=2,
                           strides=2))
    
    model.add(Conv2D(filters=64,
                     kernel_size=5,
                     strides=1,
                     padding='same',
                     input_shape=input_shape))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=2,
                           strides=2))
    
    model.add(Conv2D(filters=256,
                     kernel_size=5,
                     strides=1,
                     padding='same',
                     input_shape=input_shape))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=2,
                           strides=2))

    model.add(Flatten())
    model.add(Dense(800))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(Dropout(dropout))
    model.add(BatchNormalization())
    model.add(Dense(500))
    model.add(LeakyReLU())
    model.add(Dropout(dropout))
    model.add(BatchNormalization())
    
    # NOTE: We use identity on the last layer as we are dealing with a regression problem
    model.add(Dense(outputs, 
                    activation='linear',
                    kernel_regularizer=regularizers.l2(0.01),
                    activity_regularizer=regularizers.l1(0.01)))
    
    model.summary()
    
    return model

In [ ]:
def lenet_d_bn_l_more_filters_less_fc(input_shape, dropout=0.3, outputs=28):
    """
    Returns an uncompiled lenet_d_bn_l_more_filters, but with less nodes in the second last fc layer
    
    Parameters
    ----------
    input_shape : array-like, shape (3,)
        Tuple containing height, width and depth
    outputs : int
        Number of outputs
    
    Returns
    -------
    model : Sequential
        The uncompiled model
    """
    
    model = Sequential()

    model.add(Conv2D(filters=32,
                     kernel_size=5,
                     strides=1,
                     padding='same',
                     input_shape=input_shape))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=2,
                           strides=2))
    
    model.add(Conv2D(filters=64,
                     kernel_size=5,
                     strides=1,
                     padding='same',
                     input_shape=input_shape))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=2,
                           strides=2))
    
    model.add(Conv2D(filters=256,
                     kernel_size=5,
                     strides=1,
                     padding='same',
                     input_shape=input_shape))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=2,
                           strides=2))

    model.add(Flatten())
    model.add(Dense(400))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(Dropout(dropout))
    model.add(BatchNormalization())
    model.add(Dense(500))
    model.add(LeakyReLU())
    model.add(Dropout(dropout))
    model.add(BatchNormalization())
    
    # NOTE: We use identity on the last layer as we are dealing with a regression problem
    model.add(Dense(outputs, activation='linear'))
    
    model.summary()
    
    return model

In [ ]:
def lenet_d_bn_l_mf_2(input_shape, dropout=0.3, outputs=28):
    """
    Returns an uncompiled lenet_d_bn_l_more_filters, but with more filters
    
    Parameters
    ----------
    input_shape : array-like, shape (3,)
        Tuple containing height, width and depth
    outputs : int
        Number of outputs
    
    Returns
    -------
    model : Sequential
        The uncompiled model
    """
    
    model = Sequential()

    model.add(Conv2D(filters=64,
                     kernel_size=5,
                     strides=1,
                     padding='same',
                     input_shape=input_shape))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=2,
                           strides=2))
    
    model.add(Conv2D(filters=128,
                     kernel_size=5,
                     strides=1,
                     padding='same',
                     input_shape=input_shape))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=2,
                           strides=2))
    
    model.add(Conv2D(filters=512,
                     kernel_size=5,
                     strides=1,
                     padding='same',
                     input_shape=input_shape))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=2,
                           strides=2))

    model.add(Flatten())
    model.add(Dense(800))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(Dropout(dropout))
    model.add(BatchNormalization())
    model.add(Dense(500))
    model.add(LeakyReLU())
    model.add(Dropout(dropout))
    model.add(BatchNormalization())
    
    # NOTE: We use identity on the last layer as we are dealing with a regression problem
    model.add(Dense(outputs, activation='linear'))
    
    model.summary()
    
    return model

In [ ]:
def lenet_pow_2(input_shape, dropout=0.3, outputs=28):
    """
    Returns an uncompiled lenet_d_bn_l_more_filters, but nodes increasing by the power of 2
    
    Parameters
    ----------
    input_shape : array-like, shape (3,)
        Tuple containing height, width and depth
    outputs : int
        Number of outputs
    
    Returns
    -------
    model : Sequential
        The uncompiled model
    """
    
    model = Sequential()

    model.add(Conv2D(filters=32,
                     kernel_size=5,
                     strides=1,
                     padding='same',
                     input_shape=input_shape))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=2,
                           strides=2))
    
    model.add(Conv2D(filters=64,
                     kernel_size=5,
                     strides=1,
                     padding='same',
                     input_shape=input_shape))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=2,
                           strides=2))
    
    model.add(Conv2D(filters=128,
                     kernel_size=5,
                     strides=1,
                     padding='same',
                     input_shape=input_shape))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=2,
                           strides=2))

    model.add(Flatten())
    model.add(Dense(256))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(Dropout(dropout))
    model.add(BatchNormalization())
    model.add(Dense(512))
    model.add(LeakyReLU())
    model.add(Dropout(dropout))
    model.add(BatchNormalization())
    
    # NOTE: We use identity on the last layer as we are dealing with a regression problem
    model.add(Dense(outputs, activation='linear'))
    
    model.summary()
    
    return model

In [ ]:
def lenet_pow_2_k_4(input_shape, dropout=0.3, outputs=28):
    """
    Returns an uncompiled lenet_pow_2, but with kernel size of 4
    
    Parameters
    ----------
    input_shape : array-like, shape (3,)
        Tuple containing height, width and depth
    outputs : int
        Number of outputs
    
    Returns
    -------
    model : Sequential
        The uncompiled model
    """
    
    model = Sequential()

    model.add(Conv2D(filters=32,
                     kernel_size=4,
                     strides=1,
                     padding='same',
                     input_shape=input_shape))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=2,
                           strides=2))
    
    model.add(Conv2D(filters=64,
                     kernel_size=4,
                     strides=1,
                     padding='same',
                     input_shape=input_shape))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=2,
                           strides=2))
    
    model.add(Conv2D(filters=128,
                     kernel_size=4,
                     strides=1,
                     padding='same',
                     input_shape=input_shape))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=2,
                           strides=2))

    model.add(Flatten())
    model.add(Dense(256))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(Dropout(dropout))
    model.add(BatchNormalization())
    model.add(Dense(512))
    model.add(LeakyReLU())
    model.add(Dropout(dropout))
    model.add(BatchNormalization())
    
    # NOTE: We use identity on the last layer as we are dealing with a regression problem
    model.add(Dense(outputs, activation='linear'))
    
    model.summary()
    
    return model

In [ ]:
def lenet_pow_2_k_4_a_1(input_shape, dropout=0.3, outputs=28):
    """
    Returns an uncompiled lenet_pow_2_k_4, but with alpha=0.1 in the leaky ReLU activations
    
    Parameters
    ----------
    input_shape : array-like, shape (3,)
        Tuple containing height, width and depth
    outputs : int
        Number of outputs
    
    Returns
    -------
    model : Sequential
        The uncompiled model
    """
    
    model = Sequential()

    model.add(Conv2D(filters=32,
                     kernel_size=4,
                     strides=1,
                     padding='same',
                     input_shape=input_shape))
    model.add(LeakyReLU(alpha=0.1))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=2,
                           strides=2))
    
    model.add(Conv2D(filters=64,
                     kernel_size=4,
                     strides=1,
                     padding='same',
                     input_shape=input_shape))
    model.add(LeakyReLU(alpha=0.1))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=2,
                           strides=2))
    
    model.add(Conv2D(filters=128,
                     kernel_size=4,
                     strides=1,
                     padding='same',
                     input_shape=input_shape))
    model.add(LeakyReLU(alpha=0.1))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=2,
                           strides=2))

    model.add(Flatten())
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.1))
    model.add(BatchNormalization())
    model.add(Dropout(dropout))
    model.add(BatchNormalization())
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Dropout(dropout))
    model.add(BatchNormalization())
    
    # NOTE: We use identity on the last layer as we are dealing with a regression problem
    model.add(Dense(outputs, activation='linear'))
    
    model.summary()
    
    return model

In [ ]:
def lenet_pow_2_k_4_double_one(input_shape, dropout=0.3, outputs=28):
    """
    Returns an uncompiled lenet_pow_2_k_4, but with kernel size of 4
    
    Parameters
    ----------
    input_shape : array-like, shape (3,)
        Tuple containing height, width and depth
    outputs : int
        Number of outputs
    
    Returns
    -------
    model : Sequential
        The uncompiled model
    """
    
    model = Sequential()

    model.add(Conv2D(filters=32,
                     kernel_size=4,
                     strides=1,
                     padding='same',
                     input_shape=input_shape))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(Conv2D(filters=32,
                     kernel_size=4,
                     strides=1,
                     padding='same',
                     input_shape=input_shape))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=2,
                           strides=2))
    
    model.add(Conv2D(filters=64,
                     kernel_size=4,
                     strides=1,
                     padding='same',
                     input_shape=input_shape))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=2,
                           strides=2))
    
    model.add(Conv2D(filters=128,
                     kernel_size=4,
                     strides=1,
                     padding='same',
                     input_shape=input_shape))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=2,
                           strides=2))

    model.add(Flatten())
    model.add(Dense(256))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(Dropout(dropout))
    model.add(BatchNormalization())
    model.add(Dense(512))
    model.add(LeakyReLU())
    model.add(Dropout(dropout))
    model.add(BatchNormalization())
    
    # NOTE: We use identity on the last layer as we are dealing with a regression problem
    model.add(Dense(outputs, activation='linear'))
    
    model.summary()
    
    return model

In [ ]:
def lenet_pow_2_k_4_double_all(input_shape, dropout=0.3, outputs=28):
    """
    Returns an uncompiled lenet_pow_2_k_4, but with kernel size of 4
    
    Parameters
    ----------
    input_shape : array-like, shape (3,)
        Tuple containing height, width and depth
    outputs : int
        Number of outputs
    
    Returns
    -------
    model : Sequential
        The uncompiled model
    """
    
    model = Sequential()

    model.add(Conv2D(filters=32,
                     kernel_size=4,
                     strides=1,
                     padding='same',
                     input_shape=input_shape))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(Conv2D(filters=32,
                     kernel_size=4,
                     strides=1,
                     padding='same',
                     input_shape=input_shape))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=2,
                           strides=2))
    
    model.add(Conv2D(filters=64,
                     kernel_size=4,
                     strides=1,
                     padding='same',
                     input_shape=input_shape))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(Conv2D(filters=64,
                     kernel_size=4,
                     strides=1,
                     padding='same',
                     input_shape=input_shape))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=2,
                           strides=2))
    
    model.add(Conv2D(filters=128,
                     kernel_size=4,
                     strides=1,
                     padding='same',
                     input_shape=input_shape))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(Conv2D(filters=128,
                     kernel_size=4,
                     strides=1,
                     padding='same',
                     input_shape=input_shape))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=2,
                           strides=2))

    model.add(Flatten())
    model.add(Dense(256))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(Dropout(dropout))
    model.add(BatchNormalization())
    model.add(Dense(512))
    model.add(LeakyReLU())
    model.add(Dropout(dropout))
    model.add(BatchNormalization())
    
    # NOTE: We use identity on the last layer as we are dealing with a regression problem
    model.add(Dense(outputs, activation='linear'))
    
    model.summary()
    
    return model

In [ ]:
def lenet_pow_2_k_4_large_fc(input_shape, dropout=0.3, outputs=28):
    """
    Returns an uncompiled lenet_pow_2_k_4, but with large fully connectors
    
    Parameters
    ----------
    input_shape : array-like, shape (3,)
        Tuple containing height, width and depth
    outputs : int
        Number of outputs
    
    Returns
    -------
    model : Sequential
        The uncompiled model
    """
    
    model = Sequential()

    model.add(Conv2D(filters=32,
                     kernel_size=4,
                     strides=1,
                     padding='same',
                     input_shape=input_shape))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=2,
                           strides=2))
    
    model.add(Conv2D(filters=64,
                     kernel_size=4,
                     strides=1,
                     padding='same',
                     input_shape=input_shape))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=2,
                           strides=2))
    
    model.add(Conv2D(filters=128,
                     kernel_size=4,
                     strides=1,
                     padding='same',
                     input_shape=input_shape))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=2,
                           strides=2))

    model.add(Flatten())
    model.add(Dense(1000))
    model.add(LeakyReLU())
    model.add(BatchNormalization())
    model.add(Dropout(dropout))
    model.add(BatchNormalization())
    model.add(Dense(1000))
    model.add(LeakyReLU())
    model.add(Dropout(dropout))
    model.add(BatchNormalization())
    
    # NOTE: We use identity on the last layer as we are dealing with a regression problem
    model.add(Dense(outputs, activation='linear'))
    
    model.summary()
    
    return model

In [ ]:
lenet_model = lenet(imgs_train.shape[1:])
lenet_model = compile_model(lenet_model)
lenet_model, lenet_history = fit_model(lenet_model, 'lenet')
lenet_mse = plot_history(lenet_history)
del(lenet_model)
gc.collect()

In [ ]:
alexnet_model = alexnet(imgs_train.shape[1:])
alexnet_model = compile_model(alexnet_model)
alexnet_model, alexnet_history = fit_model(alexnet_model, 'alexnet')
alexnet_mse = plot_history(alexnet_history)
del(alexnet_model)
gc.collect()

In [ ]:
lenet_double_model = lenet_double(imgs_train.shape[1:])
lenet_double_model = compile_model(lenet_double_model)
lenet_double_model, lenet_double_history = fit_model(lenet_double_model, 'lenet_double')
lenet_double_mse = plot_history(lenet_double_history)
del(lenet_double_model)
gc.collect()

In [ ]:
lenet_double_batch_normalization_model = lenet_double_batch_normalization(imgs_train.shape[1:])
lenet_double_batch_normalization_model = compile_model(lenet_double_batch_normalization_model)
lenet_double_batch_normalization_model, lenet_double_batch_normalization_history = fit_model(lenet_double_batch_normalization_model, 'lenet_double_batch_normalization')
lenet_double_batch_normalization_mse = plot_history(lenet_double_batch_normalization_history)
del(lenet_double_batch_normalization_model)
gc.collect()

In [ ]:
lenet_double_batch_norm_high_dropout_model = lenet_double_batch_normalization(imgs_train.shape[1:], dropout=0.5)
lenet_double_batch_norm_high_dropout_model = compile_model(lenet_double_batch_norm_high_dropout_model)
lenet_double_batch_norm_high_dropout_model, lenet_double_batch_norm_high_dropout_history = fit_model(lenet_double_batch_norm_high_dropout_model, 'lenet_double_batch_norm_high_dropout')
lenet_double_batch_norm_high_dropout_mse = plot_history(lenet_double_batch_norm_high_dropout_history)
del(lenet_double_batch_norm_high_dropout_model)
gc.collect()

In [ ]:
lenet_double_bn_leaky_high_do_model = lenet_double_bn_leaky(imgs_train.shape[1:])
lenet_double_bn_leaky_high_do_model = compile_model(lenet_double_bn_leaky_high_do_model)
lenet_double_bn_leaky_high_do_model, lenet_double_bn_leaky_high_do_history = fit_model(lenet_double_bn_leaky_high_do_model, 'lenet_bn_leaky_high_do')
lenet_double_bn_leaky_high_do_mse = plot_history(lenet_double_bn_leaky_high_do_history)
del(lenet_double_bn_leaky_high_do_model)
gc.collect()

In [ ]:
lenet_d_bn_l_more_filters_high_do_model = lenet_d_bn_l_more_filters(imgs_train.shape[1:])
lenet_d_bn_l_more_filters_high_do_model = compile_model(lenet_d_bn_l_more_filters_high_do_model)
lenet_d_bn_l_more_filters_high_do_model, lenet_d_bn_l_more_filters_high_do_history = fit_model(lenet_d_bn_l_more_filters_high_do_model, 'lenet_d_bn_l_more_filters_high_do')
lenet_d_bn_l_more_filters_high_do_mse = plot_history(lenet_d_bn_l_more_filters_high_do_history)
del(lenet_d_bn_l_more_filters_high_do_model)
gc.collect()

In [ ]:
lenet_d_bn_l_more_filters_high_do_high_batch_model = lenet_d_bn_l_more_filters(imgs_train.shape[1:])
lenet_d_bn_l_more_filters_high_do_high_batch_model = compile_model(lenet_d_bn_l_more_filters_high_do_high_batch_model)
lenet_d_bn_l_more_filters_high_do_high_batch_model, lenet_d_bn_l_more_filters_high_do_high_batch_history = fit_model(lenet_d_bn_l_more_filters_high_do_high_batch_model, 'lenet_d_bn_l_more_filters_high_do_high_batch', batch_size=128)
lenet_d_bn_l_more_filters_high_do_high_batch_mse = plot_history(lenet_d_bn_l_more_filters_high_do_high_batch_history)
del(lenet_d_bn_l_more_filters_high_do_high_batch_model)
gc.collect()

In [ ]:
lenet_d_bn_l_more_filters_reg_high_do_model = lenet_d_bn_l_more_filters_reg(imgs_train.shape[1:])
lenet_d_bn_l_more_filters_reg_high_do_model = compile_model(lenet_d_bn_l_more_filters_reg_high_do_model)
lenet_d_bn_l_more_filters_reg_high_do_model, lenet_d_bn_l_more_filters_reg_high_do_history = fit_model(lenet_d_bn_l_more_filters_reg_high_do_model, 'lenet_d_bn_l_more_filters_reg_high_do')
lenet_d_bn_l_more_filters_reg_high_do_mse = plot_history(lenet_d_bn_l_more_filters_reg_high_do_history)
del(lenet_d_bn_l_more_filters_reg_high_do_model)
gc.collect()

In [ ]:
lenet_d_bn_l_more_filters_less_fc_high_do_model = lenet_d_bn_l_more_filters_less_fc(imgs_train.shape[1:])
lenet_d_bn_l_more_filters_less_fc_high_do_model = compile_model(lenet_d_bn_l_more_filters_less_fc_high_do_model)
lenet_d_bn_l_more_filters_less_fc_high_do_model, lenet_d_bn_l_more_filters_less_fc_high_do_history = fit_model(lenet_d_bn_l_more_filters_less_fc_high_do_model, 'lenet_d_bn_l_more_filters_less_fc_high_do')
lenet_d_bn_l_more_filters_less_fc_high_do_mse = plot_history(lenet_d_bn_l_more_filters_less_fc_high_do_history)
del(lenet_d_bn_l_more_filters_less_fc_high_do_model)
gc.collect()

In [ ]:
lenet_d_bn_l_mf_2_high_do_model = lenet_d_bn_l_mf_2(imgs_train.shape[1:])
lenet_d_bn_l_mf_2_high_do_model = compile_model(lenet_d_bn_l_mf_2_high_do_model)
lenet_d_bn_l_mf_2_high_do_model, lenet_d_bn_l_mf_2_high_do_history = fit_model(lenet_d_bn_l_mf_2_high_do_model, 'lenet_d_bn_l_mf_2_high_do')
lenet_d_bn_l_mf_2_high_do_mse = plot_history(lenet_d_bn_l_mf_2_high_do_history)
del(lenet_d_bn_l_mf_2_high_do_model)
gc.collect()

In [ ]:
lenet_pow_2_high_do_model = lenet_pow_2(imgs_train.shape[1:])
lenet_pow_2_high_do_model = compile_model(lenet_pow_2_high_do_model)
lenet_pow_2_high_do_model, lenet_pow_2_high_do_history = fit_model(lenet_pow_2_high_do_model, 'lenet_pow_2_high_do')
lenet_pow_2_high_do_mse = plot_history(lenet_pow_2_high_do_history)
del(lenet_pow_2_high_do_model)
gc.collect()

In [ ]:
lenet_pow_2_k_4_high_do_model = lenet_pow_2_k_4(imgs_train.shape[1:])
lenet_pow_2_k_4_high_do_model = compile_model(lenet_pow_2_k_4_high_do_model)
lenet_pow_2_k_4_high_do_model, lenet_pow_2_k_4_high_do_history = fit_model(lenet_pow_2_k_4_high_do_model, 'lenet_pow_2_k_4_high_do')
lenet_pow_2_k_4_high_do_mse = plot_history(lenet_pow_2_k_4_high_do_history)
del(lenet_pow_2_k_4_high_do_model)
gc.collect()

In [ ]:
lenet_pow_2_k_4_a_1_high_do_model = lenet_pow_2_k_4_a_1(imgs_train.shape[1:])
lenet_pow_2_k_4_a_1_high_do_model = compile_model(lenet_pow_2_k_4_a_1_high_do_model)
lenet_pow_2_k_4_a_1_high_do_model, lenet_pow_2_k_4_a_1_high_do_history = fit_model(lenet_pow_2_k_4_a_1_high_do_model, 'lenet_pow_2_k_4_a_1_high_do')
lenet_pow_2_k_4_a_1_high_do_mse = plot_history(lenet_pow_2_k_4_a_1_high_do_history)
del(lenet_pow_2_k_4_a_1_high_do_model)
gc.collect()

In [ ]:
lenet_pow_2_k_4_double_one_high_do_model = lenet_pow_2_k_4_double_one(imgs_train.shape[1:])
lenet_pow_2_k_4_double_one_high_do_model = compile_model(lenet_pow_2_k_4_double_one_high_do_model)
lenet_pow_2_k_4_double_one_high_do_model, lenet_pow_2_k_4_double_one_high_do_history = fit_model(lenet_pow_2_k_4_double_one_high_do_model, 'lenet_pow_2_k_4_double_one_high_do', patience=10)
lenet_pow_2_k_4_double_one_high_do_mse = plot_history(lenet_pow_2_k_4_double_one_high_do_history)
del(lenet_pow_2_k_4_double_one_high_do_model)
gc.collect()

In [ ]:
lenet_pow_2_k_4_double_all_high_do_model = lenet_pow_2_k_4_double_all(imgs_train.shape[1:])
lenet_pow_2_k_4_double_all_high_do_model = compile_model(lenet_pow_2_k_4_double_all_high_do_model)
lenet_pow_2_k_4_double_all_high_do_model, lenet_pow_2_k_4_double_all_high_do_history = fit_model(lenet_pow_2_k_4_double_all_high_do_model, 'lenet_pow_2_k_4_double_all_high_do', patience=10)
lenet_pow_2_k_4_double_all_high_do_mse = plot_history(lenet_pow_2_k_4_double_all_high_do_history)
del(lenet_pow_2_k_4_double_all_high_do_model)
gc.collect()

In [ ]:
x_train_gray = rgb2gray(x_train)[..., np.newaxis]
x_val_gray = rgb2gray(x_val)[..., np.newaxis]

In [ ]:
lenet_pow_2_k_4_high_do_gray_model = lenet_pow_2_k_4(x_train_gray.shape[1:])
lenet_pow_2_k_4_high_do_gray_model = compile_model(lenet_pow_2_k_4_high_do_gray_model)
lenet_pow_2_k_4_high_do_gray_model, lenet_pow_2_k_4_high_do_gray_history = \
    fit_model(lenet_pow_2_k_4_high_do_gray_model, 
              'lenet_pow_2_k_4_high_do_gray',
              x_train_=x_train_gray,
              x_val_=x_val_gray, 
              patience=10)
lenet_pow_2_k_4_high_do_gray_mse = plot_history(lenet_pow_2_k_4_high_do_gray_history)
del(lenet_pow_2_k_4_high_do_gray_model)
gc.collect()

In [ ]:
lenet_pow_2_k_4_large_fc_high_do_model = lenet_pow_2_k_4_large_fc(imgs_train.shape[1:])
lenet_pow_2_k_4_large_fc_high_do_model = compile_model(lenet_pow_2_k_4_large_fc_high_do_model)
lenet_pow_2_k_4_large_fc_high_do_model, lenet_pow_2_k_4_large_fc_high_do_history = fit_model(lenet_pow_2_k_4_large_fc_high_do_model, 'lenet_pow_2_k_4_large_fc_high_do')
lenet_pow_2_k_4_large_fc_high_do_mse = plot_history(lenet_pow_2_k_4_large_fc_high_do_history)
del(lenet_pow_2_k_4_large_fc_high_do_model)
gc.collect()

In [ ]:
results = {'lenet_mse': lenet_mse,
           'alexnet_mse': alexnet_mse,
           'lenet_double_mse': lenet_double_mse,
           'lenet_double_batch_normalization_mse': lenet_double_batch_normalization_mse,
           'lenet_double_batch_norm_high_dropout_mse': lenet_double_batch_norm_high_dropout_mse,
           'lenet_double_bn_leaky_high_do_mse': lenet_double_bn_leaky_high_do_mse,
           'lenet_d_bn_l_more_filters_high_do_mse': lenet_d_bn_l_more_filters_high_do_mse,
           'lenet_d_bn_l_more_filters_high_do_high_batch_mse': lenet_d_bn_l_more_filters_high_do_high_batch_mse,
           'lenet_d_bn_l_more_filters_reg_high_do_mse': lenet_d_bn_l_more_filters_reg_high_do_mse,
           'lenet_d_bn_l_more_filters_less_fc_high_do_mse': lenet_d_bn_l_more_filters_less_fc_high_do_mse,
           'lenet_d_bn_l_mf_2_high_do_mse': lenet_d_bn_l_mf_2_high_do_mse,
           'lenet_pow_2_high_do_mse': lenet_pow_2_high_do_mse,
           'lenet_pow_2_k_4_high_do_mse': lenet_pow_2_k_4_high_do_mse,
           'lenet_pow_2_k_4_a_1_high_do_mse': lenet_pow_2_k_4_a_1_high_do_mse,
           'lenet_pow_2_k_4_double_one_high_do_mse': lenet_pow_2_k_4_double_one_high_do_mse,
           'lenet_pow_2_k_4_double_all_high_do_mse': lenet_pow_2_k_4_double_all_high_do_mse,
           'lenet_pow_2_k_4_high_do_gray_mse': lenet_pow_2_k_4_high_do_gray_mse,
           'lenet_pow_2_k_4_large_fc_high_do_mse': lenet_pow_2_k_4_large_fc_high_do_mse
          }

results = dict(sorted(results.items(), key=lambda kv: kv[1]))

In [ ]:
fig, ax = plt.subplots()
ax.bar(results.keys(), results.values())
ax.set_ylabel('Mean squared error')
ax.set_xlabel('Model')
ax.grid(True)
for tick in ax.get_xticklabels():
    tick.set_rotation(90)

### Visualize results

Now visualize neural network results on several images from validation sample. Make sure that your network outputs different points for images (i.e. it doesn't output some constant).

> **NOTE**: It would be most appropriate to visualize on a test set as we can overfit to the validation set by trying out different architectures

In [ ]:
lenet_pow_2_k_4_high_do_model = lenet_pow_2_k_4(imgs_train.shape[1:])
lenet_pow_2_k_4_high_do_model = compile_model(lenet_pow_2_k_4_high_do_model)
model, _ = fit_model(lenet_pow_2_k_4_high_do_model, 'lenet_pow_2_k_4_high_do')

In [ ]:
# Take 10 random images from the validation set
indices = random.sample(range(x_val.shape[0]), 10)

test = []
for ind in indices:
    # NOTE: We are keeping the original dimension with ind:ind+1
    predictions = model.predict(x_val[ind:ind+1, ...])[0]
    test.append(predictions)
    visualize_points(x_val[ind, ...]*255, predictions)